# Objectives:

1. Implement data preprocessing to summarize the features in the time series. The features hypothesized to be useful are as follows:
    * Band Spectrum Energy (eeg.get_power) (for EEG)
    * Fractal Dimension of Time Series (pyEEG)
    * Skewness and Kurtosis of the distributions (scipy.stats)
    * Regular stats (mean, median, max, min of time series)
    
2. Models to train:
    * Linear Chain Conditional Random Field with structured SVM solver with a 1-slack QP with L1 slack penalty (pystruct)
    * A simple SVM like Andreas (sklearn SVC)
    * A Hidden Markov model?
    
3. Cross Validate:
    * Build a proper function/system for leave-one-subject-out CV
    * Test models and ensembles on it


In [81]:
import importlib
import utils
importlib.reload(utils)
import utils
from utils import *

In [2]:
xtrain_eeg1, xtrain_eeg2, xtrain_emg, ytrain, xtest_eeg1, xtest_eeg2, xtest_emg = load_data()

Loading xtrain...
Shapes: (64800, 512) (64800, 512) (64800, 512)
Loading ytrain...
Shape: (64800, 1)
Loading xtest...
Shapes: (43200, 512) (43200, 512) (43200, 512)


In [82]:
# CRF Model Preprocessing
eeg1_ = process_EEG(xtrain_eeg1)
eeg2_ = process_EEG(xtrain_eeg2)
emg_ = process_EMG(xtrain_emg)
xtrain_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)

eeg1_ = process_EEG(xtest_eeg1)
eeg2_ = process_EEG(xtest_eeg2)
emg_ = process_EMG(xtest_emg)
xtest_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)


/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:556: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """VECTORIZED!!! TESTED, matches the 1d time series output. Now accepts (nxd) matrices as input
/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:239: RuntimeWarning: divide by zero encountered in power
  np.sum(np.power(sig, 2), axis=1)
/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:556: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pas

# Model Implementation: Conditional 

In [ ]:
idx = 21600*2
xtrain_eeg1_ = xtrain_eeg1[0:idx]
xtrain_eeg2_ = xtrain_eeg2[0:idx]
xtrain_emg_ = xtrain_emg[0:idx]
eeg1_ = process_EEG(xtrain_eeg1_)
eeg2_ = process_EEG(xtrain_eeg2_)
emg_ = process_EMG(xtrain_emg_)
xtrain_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)
# ytrain_classes = ytrain[0:idx]['y']
ytrain_classes = ytrain.values[0:idx]
ytrain_ = ytrain.values[0:idx]

xtest_eeg1_ = xtrain_eeg1[idx:-1]
xtest_eeg2_ = xtrain_eeg2[idx:-1]
xtest_emg_ = xtrain_emg[idx:-1]
eeg1_ = process_EEG(xtest_eeg1_)
eeg2_ = process_EEG(xtest_eeg2_)
emg_ = process_EEG(xtest_emg_)
xtest_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)
ytest_ = ytrain.values[idx:-1]

# xtest_eeg1_ = xtrain_eeg1[idx:-1]
# xtest_eeg2_ = xtrain_eeg2[idx:-1]
# xtest_final = 

In [ ]:
# CRF

# xtrain_ = np.reshape(xtrain_, (xtrain_.shape[0], 1, xtrain_[0].shape[0])) # Reshape so that it works with CRF
xtrain_crf = np.reshape(xtrain_, (2, -1, xtrain_.shape[1])) # Reshape so that it works with CRF
ytrain_crf = np.reshape(ytrain_, (2, -1)) -1 # Reshape so that it works with CRF
# X_test_crf = X_test.reshape(2, -1, 49)
print(xtrain_crf.shape, ytrain_crf.shape)

print("Starting CRF...")
classes = np.array([1, 2, 3])
lmao = np.reshape(ytrain_classes, (-1,))
weights_crf = compute_class_weight("balanced", list(classes), list(lmao))
weights_crf[0] = weights_crf[0]+2.5
weights_crf[1] = weights_crf[1]+1.5
    
model = ChainCRF(class_weight=weights_crf)
ssvm = OneSlackSSVM(model=model, C=0.5, max_iter=2000)
    
ssvm.fit(xtrain_crf, ytrain_crf)   
   

# Test on the third guy
xtest_crf = np.reshape(xtest_, (1, -1, xtest_.shape[1]))
ytest_crf = np.reshape(ytest_, (1, -1)) -1
print(xtest_crf.shape, ytest_crf.shape)
y_pred_crf = ssvm.predict(xtest_crf)
y_pred_crf = np.asarray(y_pred_crf).reshape(-1) + 1  

print("BMAC:", sklearn.metrics.balanced_accuracy_score(ytest_, y_pred_crf))


# y_pred_filtered_crf = medfilt(y_pred_crf, FILTER_WINDOW)


# print(f"CRF unique predictions {np.unique(y_pred_filtered_crf)}")


# CRF Simply run model on CV

In [31]:
res = losocv_CRF_prepro(xtrain_, ytrain, C=0.1, weight_shift=0.01, max_iter=2000, fs=128)
print(res)

# Dec 18, c=0.1, w = 0, simple EMG feats, 0.94 avg. with weight_shift = 1, the thing actually converges, else it does not. 

Iteration, result: 0 0.9189317418188776
Iteration, result: 1 0.9634650343180367
Iteration, result: 2 0.9645488907325678
[0.9189317418188776, 0.9634650343180367, 0.9645488907325678]


# CRF Submission

In [65]:
pred = CRF_pred_prepro(xtrain_s, ytrain_s, xtest_s, C=0.1, weight_shift=0.01, max_iter=1000, fs=128)
make_submission('CRFc0.1w0m2000_subj3pred.csv', pred)


ValueError: Length of values does not match length of index

# CRF Grid Search

In [83]:
Cs = [0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 5.0]
weight_shifts = [0.01]
max_iters = [10, 100, 250, 500, 800, 1000]
total_res = []
for c, w, max_iter in itertools.product(Cs, weight_shifts, max_iters):
    res = losocv_CRF_prepro(xtrain_, ytrain, C=c, weight_shift=w, max_iter=max_iter, fs=128)
    total_res.append(res)
    print("C:", c, ", w:", w, ", maxiter:", max_iter, ", BMAC (Mean, Std, full):", (np.mean(res), np.std(res), res))        
    print("********************************")

# Res December 14th, Cs and weight shift, EEG and EMG running process_EMG .
# C: 0.1 , w: 0 , BMAC (Mean, Std): (0.915638922375751, 0.05039793710589233)

# Dec 18
# C: 0.1 , w: 0 , BMAC (Mean, Std, full): (0.8105643639593416, 0.1823532025093181, [0.9057309406411344, 0.9705540211342516, 0.5554081301026389])
# C: 0.2 , w: 0 , BMAC (Mean, Std, full): (0.8320971763898081, 0.15734031410977364, [0.9146093715452001, 0.9698041054523155, 0.6118780521719084])

# Dec 18 grid search with EMG running process_EEG
# C: 0.6 , w: 1 , BMAC (Mean, Std, full): (0.9515538209028076, 0.016857413648390978, [0.92784839763847, 0.9655970286576371, 0.9612160364123156])

# Dec 18 max_iter grid search
# Iteration, result: 0 0.3843139252584899
# Iteration, result: 1 0.43185969132129837
# Iteration, result: 2 0.3727368659024454
# C: 0.05 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.39630349416074456, 0.0255824085812038, [0.3843139252584899, 0.43185969132129837, 0.3727368659024454])
# ********************************
# Iteration, result: 0 0.8545253283811184
# Iteration, result: 1 0.9279638517659482
# Iteration, result: 2 0.9405287843198415
# C: 0.05 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.9076726548223027, 0.037929303653027345, [0.8545253283811184, 0.9279638517659482, 0.9405287843198415])
# ********************************
# Iteration, result: 0 0.9082440134352735
# Iteration, result: 1 0.947147076601729
# Iteration, result: 2 0.9458020054289396
# C: 0.05 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9337310318219808, 0.018030407349774995, [0.9082440134352735, 0.947147076601729, 0.9458020054289396])
# ********************************
# Iteration, result: 0 0.9079745357325523
# Iteration, result: 1 0.9466763536193374
# Iteration, result: 2 0.948981772088117
# C: 0.05 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.9345442204800022, 0.018811164225236095, [0.9079745357325523, 0.9466763536193374, 0.948981772088117])
# ********************************
# Iteration, result: 0 0.9079745357325523
# Iteration, result: 1 0.9466763536193374
# Iteration, result: 2 0.948981772088117
# C: 0.05 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9345442204800022, 0.018811164225236095, [0.9079745357325523, 0.9466763536193374, 0.948981772088117])
# ********************************
# Iteration, result: 0 0.9079745357325523
# Iteration, result: 1 0.9466763536193374
# Iteration, result: 2 0.948981772088117
# C: 0.05 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9345442204800022, 0.018811164225236095, [0.9079745357325523, 0.9466763536193374, 0.948981772088117])
# ********************************
# Iteration, result: 0 0.38428426926797976
# Iteration, result: 1 0.43185969132129837
# Iteration, result: 2 0.3727368659024454
# C: 0.1 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.3962936088305746, 0.025587044885049655, [0.38428426926797976, 0.43185969132129837, 0.3727368659024454])
# ********************************
# Iteration, result: 0 0.8125744342920846
# Iteration, result: 1 0.6841696169835045
# Iteration, result: 2 0.868353673172657
# C: 0.1 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.7883659081494154, 0.07711671324286083, [0.8125744342920846, 0.6841696169835045, 0.868353673172657])
# ********************************
# Iteration, result: 0 0.91535561097184
# Iteration, result: 1 0.9611156384577256
# Iteration, result: 2 0.9571510343473614
# C: 0.1 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9445407612589757, 0.020700390807905044, [0.91535561097184, 0.9611156384577256, 0.9571510343473614])
# ********************************
# Iteration, result: 0 0.9173226358874063
# Iteration, result: 1 0.9547583809426654
# Iteration, result: 2 0.9575831838587567
# C: 0.1 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.9432214002296094, 0.018349466363147493, [0.9173226358874063, 0.9547583809426654, 0.9575831838587567])
# ********************************
# Iteration, result: 0 0.9173226358874063
# Iteration, result: 1 0.9547583809426654
# Iteration, result: 2 0.9575831838587567
# C: 0.1 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9432214002296094, 0.018349466363147493, [0.9173226358874063, 0.9547583809426654, 0.9575831838587567])
# ********************************
# Iteration, result: 0 0.9173226358874063
# Iteration, result: 1 0.9547583809426654
# Iteration, result: 2 0.9575831838587567
# C: 0.1 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9432214002296094, 0.018349466363147493, [0.9173226358874063, 0.9547583809426654, 0.9575831838587567])
# ********************************
# Iteration, result: 0 0.38428426926797976
# Iteration, result: 1 0.43185969132129837
# Iteration, result: 2 0.3727368659024454
# C: 0.2 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.3962936088305746, 0.025587044885049655, [0.38428426926797976, 0.43185969132129837, 0.3727368659024454])
# ********************************
# Iteration, result: 0 0.9085947956269015
# Iteration, result: 1 0.7384838174202963
# Iteration, result: 2 0.9532534750200276
# C: 0.2 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.8667773626890751, 0.09253116494640658, [0.9085947956269015, 0.7384838174202963, 0.9532534750200276])
# ********************************
# Iteration, result: 0 0.9254577708060222
# Iteration, result: 1 0.9650054239802469
# Iteration, result: 2 0.96066980903715
# C: 0.2 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9503776679411398, 0.01770970250366222, [0.9254577708060222, 0.9650054239802469, 0.96066980903715])
# ********************************
# Iteration, result: 0 0.9240075579127357
# Iteration, result: 1 0.9629387386786116
# Iteration, result: 2 0.9592762662759097
# C: 0.2 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.9487408542890856, 0.017552879882522836, [0.9240075579127357, 0.9629387386786116, 0.9592762662759097])
# ********************************
# Iteration, result: 0 0.9239779019222256
# Iteration, result: 1 0.9629387386786116
# Iteration, result: 2 0.9592762662759097
# C: 0.2 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9487309689589156, 0.017566809078814704, [0.9239779019222256, 0.9629387386786116, 0.9592762662759097])
# ********************************
# Iteration, result: 0 0.9239779019222256
# Iteration, result: 1 0.9629387386786116
# Iteration, result: 2 0.9592762662759097
# C: 0.2 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9487309689589156, 0.017566809078814704, [0.9239779019222256, 0.9629387386786116, 0.9592762662759097])
# ********************************
# Iteration, result: 0 0.38428426926797976
# Iteration, result: 1 0.43185969132129837
# Iteration, result: 2 0.3727368659024454
# C: 0.4 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.3962936088305746, 0.025587044885049655, [0.38428426926797976, 0.43185969132129837, 0.3727368659024454])
# ********************************
# Iteration, result: 0 0.917877345513844
# Iteration, result: 1 0.5906687085151296
# Iteration, result: 2 0.8620469465394759
# C: 0.4 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.7901976668561499, 0.14291748732491, [0.917877345513844, 0.5906687085151296, 0.8620469465394759])
# ********************************
# Iteration, result: 0 0.9242652759973878
# Iteration, result: 1 0.962763697815455
# Iteration, result: 2 0.9564579558650571
# C: 0.4 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9478289765593, 0.01685974660772113, [0.9242652759973878, 0.962763697815455, 0.9564579558650571])
# ********************************
# Iteration, result: 0 0.926935436731296
# Iteration, result: 1 0.9632980022360053
# Iteration, result: 2 0.9600836844424379
# C: 0.4 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.9501057078032463, 0.016436322700658177, [0.926935436731296, 0.9632980022360053, 0.9600836844424379])
# ********************************
# Iteration, result: 0 0.926784099672131
# Iteration, result: 1 0.963657873189657
# Iteration, result: 2 0.9605292556024607
# C: 0.4 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9503237428214163, 0.016693974196268068, [0.926784099672131, 0.963657873189657, 0.9605292556024607])
# ********************************
# Iteration, result: 0 0.926784099672131
# Iteration, result: 1 0.963657873189657
# Iteration, result: 2 0.9605292556024607
# C: 0.4 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9503237428214163, 0.016693974196268068, [0.926784099672131, 0.963657873189657, 0.9605292556024607])
# ********************************
# Iteration, result: 0 0.38428426926797976
# Iteration, result: 1 0.43185969132129837
# Iteration, result: 2 0.3727368659024454
# C: 0.6 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.3962936088305746, 0.025587044885049655, [0.38428426926797976, 0.43185969132129837, 0.3727368659024454])
# ********************************
# Iteration, result: 0 0.8934936819001159
# Iteration, result: 1 0.845913418238372
# Iteration, result: 2 0.615835020811267
# C: 0.6 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.7850807069832516, 0.12124093667404506, [0.8934936819001159, 0.845913418238372, 0.615835020811267])
# ********************************
# Iteration, result: 0 0.9295807039782841
# Iteration, result: 1 0.9614362579130559
# Iteration, result: 2 0.9582047629633855
# C: 0.6 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9497405749515752, 0.01431609672471852, [0.9295807039782841, 0.9614362579130559, 0.9582047629633855])
# ********************************

# Iteration, result: 0 0.9270727152127338
# Iteration, result: 1 0.9655641121830866
# Iteration, result: 2 0.9613177347749718
# C: 0.6 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.951318187390264, 0.017231562535196514, [0.9270727152127338, 0.9655641121830866, 0.9613177347749718])
# ********************************
# Iteration, result: 0 0.92784839763847
# Iteration, result: 1 0.9655970286576371
# Iteration, result: 2 0.9612160364123156
# C: 0.6 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9515538209028076, 0.016857413648390978, [0.92784839763847, 0.9655970286576371, 0.9612160364123156])
# ********************************
# Iteration, result: 0 0.92784839763847
# Iteration, result: 1 0.9655970286576371
# Iteration, result: 2 0.9612160364123156
# C: 0.6 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9515538209028076, 0.016857413648390978, [0.92784839763847, 0.9655970286576371, 0.9612160364123156])
# ********************************
# Iteration, result: 0 0.38428426926797976
# Iteration, result: 1 0.43185969132129837
# Iteration, result: 2 0.3727368659024454
# C: 0.8 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.3962936088305746, 0.025587044885049655, [0.38428426926797976, 0.43185969132129837, 0.3727368659024454])
# ********************************
# Iteration, result: 0 0.7976757421847053
# Iteration, result: 1 0.7007024008621568
# Iteration, result: 2 0.9006422210628849
# C: 0.8 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.799673454703249, 0.08163731200439714, [0.7976757421847053, 0.7007024008621568, 0.9006422210628849])
# ********************************
# Iteration, result: 0 0.8919704082035809
# Iteration, result: 1 0.948775230003226
# Iteration, result: 2 0.9572228919079491
# C: 0.8 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9326561767049187, 0.028975156793621958, [0.8919704082035809, 0.948775230003226, 0.9572228919079491])
# ********************************
# Iteration, result: 0 0.9272729728322293
# Iteration, result: 1 0.9653859616626743
# Iteration, result: 2 0.9602668516941896
# C: 0.8 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.9509752620630311, 0.01688984325569257, [0.9272729728322293, 0.9653859616626743, 0.9602668516941896])
# ********************************
# Iteration, result: 0 0.9265366010353305
# Iteration, result: 1 0.9656693419824388
# Iteration, result: 2 0.9605270519131293
# C: 0.8 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9509109983102996, 0.017362684482873845, [0.9265366010353305, 0.9656693419824388, 0.9605270519131293])
# ********************************
# Iteration, result: 0 0.9265366010353305
# Iteration, result: 1 0.9656693419824388
# Iteration, result: 2 0.9605270519131293
# C: 0.8 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9509109983102996, 0.017362684482873845, [0.9265366010353305, 0.9656693419824388, 0.9605270519131293])
# ********************************
# Iteration, result: 0 0.38428426926797976
# Iteration, result: 1 0.43185969132129837
# Iteration, result: 2 0.3727368659024454
# C: 1.0 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.3962936088305746, 0.025587044885049655, [0.38428426926797976, 0.43185969132129837, 0.3727368659024454])
# ********************************
# Iteration, result: 0 0.8631006109475473
# Iteration, result: 1 0.8020654744614749
# Iteration, result: 2 0.8088051162930762
# C: 1.0 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.8246570672340328, 0.027322582027134543, [0.8631006109475473, 0.8020654744614749, 0.8088051162930762])
# ********************************
# Iteration, result: 0 0.9047853948244021
# Iteration, result: 1 0.9656105658718929
# Iteration, result: 2 0.9547473578161595
# C: 1.0 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9417144395041515, 0.02648670204756507, [0.9047853948244021, 0.9656105658718929, 0.9547473578161595])
# ********************************
# Iteration, result: 0 0.9267370674118699
# Iteration, result: 1 0.9653359380610408
# Iteration, result: 2 0.9608336342375825
# C: 1.0 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.9509688799034978, 0.017232783435489844, [0.9267370674118699, 0.9653359380610408, 0.9608336342375825])
# ********************************
# Iteration, result: 0 0.9241461733266849
# Iteration, result: 1 0.9666668316689847
# Iteration, result: 2 0.9609692320544577
# C: 1.0 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9505940790167092, 0.018845590941175522, [0.9241461733266849, 0.9666668316689847, 0.9609692320544577])
# ********************************
# Iteration, result: 0 0.9241461733266849
# Iteration, result: 1 0.9666668316689847
# Iteration, result: 2 0.9609692320544577
# C: 1.0 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9505940790167092, 0.018845590941175522, [0.9241461733266849, 0.9666668316689847, 0.9609692320544577])
# ********************************
# C: 5.0 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.3960796696180886, 0.025689043252324906, [0.3836424516305219, 0.43185969132129837, 0.3727368659024454])
# ********************************
# C: 5.0 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.7798123538194982, 0.07144208545150875, [0.7992690272314628, 0.8559445770656623, 0.6842234571613695])
# ********************************
# C: 5.0 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.8452760102245901, 0.12264125120904101, [0.9056508253757034, 0.6742785319197694, 0.9558986733782974])
# ********************************
# C: 5.0 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.942680153159893, 0.020143667940895345, [0.9164677026200563, 0.9654472073120576, 0.9461255495475651])
# ********************************
# C: 5.0 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9441049909329459, 0.01991575248672787, [0.9180014989324746, 0.9663170630575567, 0.9479964108088064])
# ********************************
# C: 5.0 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9438453436545219, 0.019810726879900445, [0.9180049502109369, 0.9661411949814905, 0.9473898857711385])
# ********************************


Iteration, result: 0 0.3787042414760368
Iteration, result: 1 0.430491965488931
Iteration, result: 2 0.3741881130679783
C: 0.05 , w: 1 , maxiter: 10 , BMAC (Mean, Std, full): (0.39446144001098205, 0.02554405251863748, [0.3787042414760368, 0.430491965488931, 0.3741881130679783])
********************************
Iteration, result: 0 0.8599284455498336
Iteration, result: 1 0.9336212314812885
Iteration, result: 2 0.9195659613551124
C: 0.05 , w: 1 , maxiter: 100 , BMAC (Mean, Std, full): (0.9043718794620782, 0.03194580585944454, [0.8599284455498336, 0.9336212314812885, 0.9195659613551124])
********************************
Iteration, result: 0 0.9082054231010575
Iteration, result: 1 0.937789717223645
Iteration, result: 2 0.9567433399860704
C: 0.05 , w: 1 , maxiter: 250 , BMAC (Mean, Std, full): (0.9342461601035909, 0.019973314558537368, [0.9082054231010575, 0.937789717223645, 0.9567433399860704])
********************************
Iteration, result: 0 0.9073373590106315
Iteration, result: 1 0.9

Iteration, result: 0 0.9240712781759446
Iteration, result: 1 0.9674013152719255
Iteration, result: 2 0.9604571008904511
C: 0.6 , w: 1 , maxiter: 500 , BMAC (Mean, Std, full): (0.9506432314461071, 0.019001878043810008, [0.9240712781759446, 0.9674013152719255, 0.9604571008904511])
********************************
Iteration, result: 0 0.9241810358433531
Iteration, result: 1 0.9673242172417801
Iteration, result: 2 0.9606229330232748
C: 0.6 , w: 1 , maxiter: 800 , BMAC (Mean, Std, full): (0.9507093953694694, 0.01895683160770735, [0.9241810358433531, 0.9673242172417801, 0.9606229330232748])
********************************
Iteration, result: 0 0.9241810358433531
Iteration, result: 1 0.9673242172417801
Iteration, result: 2 0.9606229330232748
C: 0.6 , w: 1 , maxiter: 1000 , BMAC (Mean, Std, full): (0.9507093953694694, 0.01895683160770735, [0.9241810358433531, 0.9673242172417801, 0.9606229330232748])
********************************
Iteration, result: 0 0.3788020757421989
Iteration, result: 1 0